In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

Loading Dataset


In [ ]:
dataset = '/content/MyNews - Sheet1.csv'

In [ ]:
import chardet
with open('/content/MyNews - Sheet1.csv', 'rb') as f:
    enc = chardet.detect(f.read())

In [ ]:
data = pd.read_csv(dataset,encoding='utf-8')
print(data.head())

   index                                          Sentences
0      0  အန်ဒရီယာ မာစီ သည် အီတလီ အတွက် စမ်းသပ်မှု တစ်ခု...
1      1  ပေါ်တူဂီ သည် ဘယ်သောအခါမှ စွန့်လွှတ်မှု မရှိခဲ့...
2      2  အီတလီ သည် ပထမပိုင်း ၌ ၁၆-၅ ဖြင့် ဦးဆောင်ခဲ့ သေ...
3      3  သို့သော်လည်း ဘော်တိုလပ်စီ သည် မော်ရို ဘာဂမ်မက်...
4      4  နယူးဇီလန် သည် ရမှတ် ကွာခြားချက် အပေါ် စကော့တလန...


In [ ]:
data.isnull().sum()

index        0
Sentences    4
dtype: int64

Preprocessing

In [ ]:
stopwordslist = []
slist = []
with open("/content/stopword (1).txt", encoding = 'utf8') as stopwordsfile:
    stopwords = stopwordsfile.readlines()
    slist.extend(stopwords)
    for w in range(len(slist)):
        temp = slist[w]
        stopwordslist.append(temp.rstrip())

In [ ]:
import re
CleanPattern = re.compile(r'\d+|[၊။!-/:-@[-`{-~\t ]|[A-za-z0-9]')
def clean_sentence(sentence):
     sent = CleanPattern.sub(" ",sentence)
     return sent

def stop_word(sentence):
  new_sentence = []
  for word in sentence.split():
    if word not in stopwordslist:
      new_sentence.append(word)
  return(' '.join(new_sentence))

def process(line):
    #line = clean_sentence(line)
    sentence = stop_word(line)
    return sentence

BreakPattern = re.compile(r'(?:(?<!္)([က-ဪဿ၊-၏]|[၀-၉]+|[^က-၏]+)(?![ှျ]?[့္်]))')

def sylbreak(sentence):
    line = BreakPattern.sub(" " + r"\1",sentence)
    return line
       
def process(sentence):
    line = clean_sentence(sentence)
    sentence = stop_word(line)
    return sentence

In [ ]:
data['Sentences'] = data['Sentences'].apply(str)
data['Sentences'].apply(process)
data['Sentences'].head()

0    အန်ဒရီယာ မာစီ သည် အီတလီ အတွက် စမ်းသပ်မှု တစ်ခု...
1    ပေါ်တူဂီ သည် ဘယ်သောအခါမှ စွန့်လွှတ်မှု မရှိခဲ့...
2    အီတလီ သည် ပထမပိုင်း ၌ ၁၆-၅ ဖြင့် ဦးဆောင်ခဲ့ သေ...
3    သို့သော်လည်း ဘော်တိုလပ်စီ သည် မော်ရို ဘာဂမ်မက်...
4    နယူးဇီလန် သည် ရမှတ် ကွာခြားချက် အပေါ် စကော့တလန...
Name: Sentences, dtype: object

In [ ]:
data.isnull().sum()

index        0
Sentences    0
dtype: int64

Building the corpus

In [ ]:
def build_corpus(data):
  #data = ['Sentences']
  corpus = []
  for col in ['Sentences']:
    for sentence in data[col].iteritems():
      wordlist = sentence[1].split(" ")
      corpus.append(wordlist)
  return corpus

In [ ]:
corpus = build_corpus(data)
corpus[:10]

[['အန်ဒရီယာ',
  'မာစီ',
  'သည်',
  'အီတလီ',
  'အတွက်',
  'စမ်းသပ်မှု',
  'တစ်ခု',
  'အဖြစ်',
  'စတုတ္ထ',
  'မိနစ်',
  'တွင်',
  'အမှတ်ပေးခြင်း',
  'ကို',
  'ဖွင့်လှစ်ပေးခဲ့သည်',
  '။'],
 ['ပေါ်တူဂီ',
  'သည်',
  'ဘယ်သောအခါမှ',
  'စွန့်လွှတ်မှု',
  'မရှိခဲ့',
  'ပဲ',
  'ဒေးဗစ်',
  'ပန်နယ်ဗါ',
  'သည်',
  '၃၃',
  'မိနစ်',
  'တွင်',
  'သူတို့၏',
  'ကစားပွဲ',
  'ရဲ့',
  'အမှတ်များ',
  'ကို',
  'သာ',
  'အထောက်အကူပြု',
  'သည့်',
  'ကြိုးစားမှု',
  'တစ်ခု',
  'မှ',
  'အမှတ်ရခဲ့ပါသည်။'],
 ['အီတလီ',
  'သည်',
  'ပထမပိုင်း',
  '၌',
  '၁၆-၅',
  'ဖြင့်',
  'ဦးဆောင်ခဲ့',
  'သော်လည်း',
  'ပေါ်တူဂီ',
  'မှ',
  'ဒုတိယပိုင်း',
  'တော်တော်များများ',
  'တွင်',
  'ဂိုး',
  'အရေအတွက်',
  'ညီ',
  'စေ',
  'ခဲ့သည်',
  '။'],
 ['သို့သော်လည်း',
  'ဘော်တိုလပ်စီ',
  'သည်',
  'မော်ရို',
  'ဘာဂမ်မက်စကို',
  '၏',
  'ကြိုးစားမှုများ',
  'နှင့်',
  'အန်ဒရီယာ',
  'မာစီ',
  '၏',
  'ဒုတိယဂိုး',
  'နောက်ပိုင်း',
  'အီတလီ',
  'အတွက်',
  'အနိုင်ရရှိ',
  'စေရန်',
  'သူ၏',
  'စတုတ္ထမြောက်',
  'ကစားပွဲ',
  '၏',
  'ပယ်နယ်လ်တီ',
  '

Word2Vec Training

In [ ]:
!pip install gensim
import gensim

In [ ]:
model = gensim.models.Word2Vec(corpus, size= 32, window=3, min_count=2,iter=100)

In [ ]:
model.wv['ငြိမ်းချမ်းရေး']

array([ 0.4209406 , -1.4978193 ,  0.5399212 ,  3.1533613 , -3.8868368 ,
       -0.65335333,  2.8468697 ,  2.4747486 , -2.6933877 , -5.495833  ,
        5.8994946 , -3.8887768 ,  3.0242186 ,  0.19013084, -4.5360518 ,
       -1.7203277 ,  4.655463  , -1.8002901 , -1.0479832 ,  0.93547654,
        2.3086464 ,  0.7840426 ,  1.1169966 ,  0.289341  ,  2.1270802 ,
       -0.39438522,  0.84576756,  0.42771676, -0.6656337 ,  0.15496299,
       -3.0893462 ,  0.46750134, -0.5352471 ,  0.13679625, -2.3966954 ,
       -0.780969  ,  2.6128018 ,  0.43144763,  2.4074264 , -0.70601094,
       -1.2891293 , -5.1662717 , -0.9850791 ,  0.98597956, -0.8340979 ,
       -0.04918421, -0.96536046,  2.5116715 ,  1.9773432 ,  0.7617642 ,
       -2.4024007 ,  2.6907852 , -1.3998044 ,  1.9605647 ,  1.8911732 ,
       -2.109315  ,  0.54784817,  0.5940469 ,  2.8603823 , -0.57831466,
        0.73844945, -3.0586927 , -1.9124746 ,  1.188963  ,  0.70348954,
       -0.4311535 , -5.010654  ,  0.90715665, -4.2820153 , -0.34

In [ ]:
# !pip install gensim
# import gensim 

Similar the words

In [ ]:
model.most_similar('စီးပွားရေး')[:10]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('ပြည်သူတို့', 0.49253690242767334),
 ('ကြိုဆိုပွဲ', 0.4672148525714874),
 ('ဖွံ့ဖြိုးတိုးတက်မှု', 0.44676464796066284),
 ('ဘဏ္ဍာရေး', 0.4317590892314911),
 ('တရုတ်ပြည်သူ့', 0.4240837097167969),
 ('နိုင်ငံခြားရေး', 0.42279642820358276),
 ('မူဝါဒ', 0.4201165735721588),
 ('မထင်ရှားသော', 0.40907371044158936),
 ('တင်းမာမှုများ', 0.40880316495895386),
 ('ပေါင်းစည်းမှု', 0.4081439971923828)]

In [ ]:
model.wv.similarity("ဥပဒေ", "မူဝါဒ")

0.28067178

In [ ]:
#model.save("word2vec.model")
#model.save("model.bin")
# model.wv.save_word2vec_format('new_model.bin')
# model.wv.save_word2vec_format('test_w2v.txt', binary=False)
model.save("model.bin")

In [ ]:
from gensim.models import Word2Vec
model = Word2Vec.load('/content/word2vec.model')
vocab = list(model.wv.vocab)
print(vocab)

FileNotFoundError: ignored

In [ ]:
model.wv.similarity('လယ်ယာ','မြေ')

In [ ]:
dissimlar_words = model.doesnt_match('ပြည်ခိုင်ဖြိုးပါတီ ဥက္ကဋ္ဌ ဦးသိန်းစိန်က လက်ရှိ အစိုးရသစ် နှင့် တူညီသော လုပ်ငန်းများ တွင် ပူးပေါင်း လုပ်ဆောင်မည် ဆို သည့် ပြောကြားချက်ကို ကြိုဆို ကြောင်း သူရဦးရွှေမန်း က ပြောကြားခဲ့သည်။'.split())
print(dissimlar_words)